In [1]:
import eGNN
import data_read
import protein_residues
import torch
import torch.nn as nn
import numpy as np
import math
import pickle as pkl
from tqdm import tqdm
from Bio.SeqUtils import seq1, seq3
from Bio.PDB import PDBIO, StructureBuilder
import gc
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device("cpu")
pdb_dir ='/mnt/rna01/nico/dynamics_project/learn/eGNN/data/dompdb/'
list_path = "/mnt/rna01/nico/dynamics_project/learn/eGNN/data/clean_pdb_id.txt"

In [2]:
data_std = 10 #The std of protein coords is determined to be ~10 as indicated by data_processing.ipynb

# New Utils

In [3]:
def generate_res_object(pdb_seq,coords):
    residues = []
    for i ,aa in enumerate(pdb_seq, start=0):
        if aa == "G":
            res = {"name":seq3(aa),"atoms":[("N", coords[i,0,:].tolist()), ("CA", coords[i,1,:].tolist()), ("C", coords[i,2,:].tolist())]}
        else:
            res = {"name":seq3(aa),"atoms":[("N", coords[i,0,:].tolist()), ("CA", coords[i,1,:].tolist()), ("C", coords[i,2,:].tolist()),("CB", coords[i,3,:].tolist())]}
        residues.append(res)
    return residues

In [4]:
def generate_pdb(pdb_id,residues):
    builder = StructureBuilder.StructureBuilder()
    
    # Create a structure object
    builder.init_structure("Predicted eGNN Backbone ")
    builder.init_model(0)
    builder.init_chain("A")  # Single chain "A"
    builder.init_seg(" ")
    
    for res_id, residue in enumerate(residues, start=1):
        builder.init_residue(residue["name"], " ", res_id, " ")
    
        # Add atoms to the residue
        for atom_name, coords in residue["atoms"]:
            builder.init_atom(atom_name, coords, 1.0, 1.0, " ", atom_name, res_id, atom_name[0])

    structure = builder.get_structure()
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_id+".pdb")
    

# Objects

In [5]:
class Diffusion(nn.Module):

    def __init__(self, T, b_initial, b_final, device):
        super().__init__()
        self.T = T
        self.beta = torch.linspace(b_initial,b_final,T).to(device)
        
    def _CoMGaussNoise(self, x_t1, t1, t2):
        if t2 == 0:
            a_mul = torch.prod(1-self.beta[0])
        else:
            a_mul = torch.prod(1-self.beta[t1:t2+1])
        eps = torch.normal(mean=torch.zeros_like(x_t1),std=torch.ones_like(x_t1))
        x1_mean = torch.mean(x_t1.flatten(end_dim=-2),dim=-2, keepdim=True)[None,:]
        eps = eps - x1_mean 
        x_t2 = torch.sqrt(a_mul)*x_t1 + torch.sqrt(1-a_mul)*eps
        return x_t2, eps

    def _GaussNoise(self, x_t1, t1, t2):
        if t2 == 0:
            a_mul = torch.prod(1-self.beta[0])
        else:
            a_mul = torch.prod(1-self.beta[t1:t2+1])
        eps = torch.normal(mean=torch.zeros_like(x_t1),std=torch.ones_like(x_t1))
        x_t2 = torch.sqrt(a_mul)*x_t1 + torch.sqrt(1-a_mul)*eps
        #Same output has the same shape as the input
        return x_t2, eps
        
    def forward(self, x, h, t2):
        x_perturbed, x_eps = self._GaussNoise(x, 0, t2)
        h_perturbed, h_eps = self._GaussNoise(h,0,t2)
        return x_perturbed, x_eps, h_perturbed, h_eps

In [6]:
def input_generation(pdb_path,edge_device="cuda"):
    frames, seq = data_read.get_backbone(pdb_path)
    frames = torch.from_numpy(frames[0]); seq = seq[0]
    frames = frames.to(torch.float32)
    n = len(seq)
    seq_id = data_read.encode(seq[0])
    
#The default values as specifed by the DDPM paper(https://arxiv.org/pdf/2006.11239) are constants chosen 
#to be small relative to data scaled to [−1, 1] 
    
    #Assumes fully connected graph
    row =torch.arange(0,n).repeat_interleave(n).to(device)
    col =torch.arange(0,n).repeat(n).to(device)
    edge_id=(row,col)

    return frames, seq, edge_id

In [7]:
class prot_eGNN(nn.Module):
    
    def __init__(self, process_device, gnn_device, embed_dim, sc_dim, in_node_nf, hidden_nf, out_node_nf,
                 T, b_initial=0.0001, b_final=0.02, 
                 in_edge_nf=0, act_fn=nn.SiLU(), n_layers=6, residual=True, attention=True, normalize=False, tanh=False                 
                ):
        super().__init__()

        self.sc_dim = sc_dim
        self.process_device = process_device
        self.gnn_device = gnn_device
        self.T = T
        
        self.embedding = nn.Embedding(20,embed_dim)
        self.diffusion = Diffusion(T, b_initial=0.0001, b_final=0.02, device=self.process_device) 
        self.EGNN = eGNN.EGNN(in_node_nf, hidden_nf, out_node_nf, in_edge_nf=0, 
                              device=self.gnn_device , act_fn=nn.SiLU(), n_layers=n_layers, residual=residual, attention=attention, normalize=normalize, tanh=tanh)

    def _sc_embed(self, t):
        half_dim = self.sc_dim//2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim,) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

    def sc_pos_embed(self, h):
        t = torch.arange(0,h.shape[0])+1
        half_dim = h.shape[-1]//2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim,) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        h += emb
        return h
        
    def sample(self, seq_id):
        #seq_id is a 1D tensor
        x_t = torch.randn((seq_id.shape[0], 4, 3)).to(self.gnn_device) #N x 4 x 3
        
        #This remains constant, no need to recompute
        n_aa = seq_id.shape[0]
        seq_id=seq_id.to(self.gnn_device)
        h = self.embedding(seq_id)
        h = h.to(self.process_device)
        h = self.sc_pos_embed(h) 

        #Generate Edges
        row =torch.arange(0,n_aa).repeat_interleave(n_aa).to(self.gnn_device)
        col =torch.arange(0,n_aa).repeat(n_aa).to(self.gnn_device)
        edges_id=(row,col)
        
        for t in range(self.T-1, -1, -1):  
            if t > 0:
                z = torch.normal(mean=torch.zeros_like(x_t),std=torch.ones_like(x_t))
            else:
                z = torch.zeros_like(x_t)
            sc_emb = self._sc_embed(torch.tensor([t+1])).repeat(n_aa, 1)
            h_t = torch.cat((h,sc_emb),dim=-1)
            h_t = h_t.to(self.gnn_device)
            pred_h_eps, pred_x_eps= self.EGNN(h_t, x_t, edges_id, edge_attr=None)
            if t != 0:
                a_mul = torch.prod(1-self.diffusion.beta[0:t+1])
                a_mul_1 = torch.prod(1-self.diffusion.beta[0:t])
            else:
                a_mul = torch.prod(1-self.diffusion.beta[t])
                a_mul_1 = 0
            a = 1-self.diffusion.beta[t]
            #N.B. Vars = mul(Bt) for Xo ~ N(0,I) if Xo is a predetermined point the defined vars is prefered. See p.g.,3 of the DDPM paper
            std = ((1-a_mul_1)/(1-a_mul))*self.diffusion.beta[t]
            std= torch.sqrt(std)
            x_t = (a**(-0.5))*(x_t- ((1-a)/(torch.sqrt(1-a_mul)))*pred_x_eps)+std*z
        return x_t #Multiply the output by the global std
        
    def forward(self, x, seq_id, edges_id):
        assert x.shape[0]==seq_id.shape[0], "Sequence length and coordinate first dimension is not of the same shape!"

        n_aa = seq_id.shape[0]
        seq_id=seq_id.to(self.gnn_device)
        h = self.embedding(seq_id)
        h = h.to(self.process_device)
        h = self.sc_pos_embed(h)
        
        t2 = torch.randint(0, high=self.T, size=(1,))
        sc_emb = self._sc_embed(t2+1).repeat(n_aa, 1)
        x_perturbed, x_eps, h_perturbed, h_eps = self.diffusion(x,h,t2)
        x_perturbed=x_perturbed.to(self.gnn_device)
        h = torch.cat((h,sc_emb),dim=-1)
        h = h.to(self.gnn_device)
        #h_perturbed=h_perturbed.to(self.gnn_device)
        pred_h_eps, pred_x_eps= self.EGNN(h, x_perturbed, edges_id, edge_attr=None)
        return pred_h_eps, h_eps, pred_x_eps, x_eps


In [8]:
#Params
#Num of clean proteins 31065
steps = 300000
batch_size = 500
with open(list_path) as file:
    lines = [line.rstrip() for line in file]
np.random.seed(seed=17)
np.random.shuffle(lines)
train_list = np.array(lines[:-len(lines)//5])
val_list = np.array(lines[-len(lines)//5:])
np.savetxt("train_list.csv", train_list, delimiter=",", fmt='%s')
np.savetxt("val_list.csv", val_list, delimiter=",", fmt='%s')
#Instantiate Models 
embed_dim = 64
sc_dim = 32
in_node_nf = embed_dim + sc_dim
hidden_nf =128
out_node_nf = 64

model = prot_eGNN("cpu", "cuda" , embed_dim, sc_dim, in_node_nf, hidden_nf, out_node_nf,
                 1000, b_initial=0.0001, b_final=0.02, 
                 in_edge_nf=0, act_fn=nn.SiLU(), n_layers=5, residual=True, attention=True, normalize=False, tanh=False)
model=model.to("cuda")

In [9]:
#Define loss here
denoising_loss = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.01, lr=0.0001)

In [10]:
#Training Loop 1
g_pdb_dir="/mnt/rna01/nico/dynamics_project/learn/eGNN/generated_pdb/"
model.train()
agg_loss = torch.tensor([0.],dtype=torch.float32).to("cuda")
for i in tqdm(range(steps)):
    pdb_path = pdb_dir+np.random.choice(train_list)
    try:
        frames,seq,edges=input_generation(pdb_path)
    except:
        continue
    CoM = torch.mean(frames.flatten(end_dim=-2),dim=-2, keepdim=True)
    frames = frames - CoM[None,:]
    frames = frames/data_std
    i_seq = data_read.encode(seq)
    
    pred_h_eps, h_eps, pred_x_eps, x_eps=model(frames, i_seq, edges)

    
    x_eps = x_eps.to("cuda")
    loss = denoising_loss(pred_x_eps,x_eps)
    agg_loss +=loss
    if (i+1)%500 == 0:
        print(f"{i+1} Step : {loss}")
        print(f"{i+1} Step : {agg_loss/500}")
        print(type(agg_loss))
        agg_loss = torch.tensor([0.],dtype=torch.float32).to("cuda")
        torch.save({
            'epoch': i+1,
            'model_state_dict': model.state_dict(),
            'step_loss': agg_loss
            }, f'./model_weights/prot_eGNN_{i+1}')
        model.eval()
        if i>9999:
            for j in val_list[:30]:
                v_frames,v_seq,v_edges=input_generation(pdb_dir+j)
                del v_frames
                del v_edges
                gc.collect()
                v_i_seq = data_read.encode(v_seq)
                with torch.no_grad():
                    coords=model.sample(v_i_seq)
                    coords=coords*data_std
                res = generate_res_object(v_seq,coords)
                generate_pdb(g_pdb_dir+f"pred_{j}_model_{i}",res)  
        model.train()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  2%|█▎                                                                             | 500/30000 [00:20<25:58, 18.93it/s]

500 Step : 0.0792664960026741
500 Step : tensor([0.5158], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


  3%|██▌                                                                           | 1001/30000 [00:39<23:15, 20.77it/s]

1000 Step : 0.007118523120880127
1000 Step : tensor([0.2666], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


  5%|███▉                                                                          | 1502/30000 [00:58<17:46, 26.72it/s]

...

11000 Step : 0.12483146786689758
11000 Step : tensor([0.8734], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


 38%|█████████████████████████████▌                                               | 11499/30000 [11:33<09:01, 34.17it/s]

11500 Step : 0.06993775814771652
11500 Step : tensor([0.3539], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


 38%|█████████████████████████████▌                                               | 11499/30000 [13:45<22:08, 13.93it/s]


KeyboardInterrupt: 

# Generate Some PDB Samples

In [10]:
# Training Loop 2
g_pdb_dir="/mnt/rna01/nico/dynamics_project/learn/eGNN/generated_pdb/"
model.train()
agg_loss = torch.tensor([0.],dtype=torch.float32).to("cuda")
for i in tqdm(range(steps)):
    pdb_path = pdb_dir+np.random.choice(train_list)
    try:
        frames,seq,edges=input_generation(pdb_path)
    except:
        continue
    if len(seq)>300:continue
    CoM = torch.mean(frames.flatten(end_dim=-2),dim=-2, keepdim=True)
    frames = frames - CoM[None,:]
    frames = frames/data_std
    i_seq = data_read.encode(seq)
    
    pred_h_eps, h_eps, pred_x_eps, x_eps=model(frames, i_seq, edges)

    
    x_eps = x_eps.to("cuda")
    loss = denoising_loss(pred_x_eps,x_eps)
    loss.backward()
    agg_loss +=loss
    if (i+1)%batch_size ==0:
        optimizer.step()
        optimizer.zero_grad()
    if (i+1)%500 == 0:
        print(f"{i+1} Step : {loss}")
        print(f"{i+1} Step : {agg_loss/500}")
        print(type(agg_loss))
        torch.save({
            'epoch': i+1,
            'model_state_dict': model.state_dict(),
            'step_loss': agg_loss[0]
            }, f'./model_weights/prot_eGNN_{i+1}')
        agg_loss = torch.tensor([0.],dtype=torch.float32).to("cuda")
        if i>9999:
            model.eval()
            for j in val_list[:30]:
                v_frames,v_seq,v_edges=input_generation(pdb_dir+j)
                del v_frames
                del v_edges
                gc.collect()
                v_i_seq = data_read.encode(v_seq)
                with torch.no_grad():
                    coords=model.sample(v_i_seq)
                    coords=coords*data_std
                res = generate_res_object(v_seq,coords)
                generate_pdb(g_pdb_dir+f"pred_{j+1}_model_{i}",res)  
            model.train()

  0%|▏                                                                           | 502/300000 [00:19<3:39:01, 22.79it/s]

500 Step : 0.004494435619562864
500 Step : tensor([0.4007], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


  0%|▎                                                                          | 1001/300000 [00:38<3:52:45, 21.41it/s]

1000 Step : 0.001537179690785706
1000 Step : tensor([0.3827], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


  1%|▍                                                                          | 1502/300000 [00:57<2:56:47, 28.14it/s]

...

7500 Step : 0.00013185612624511123
7500 Step : tensor([0.4135], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


  3%|██                                                                         | 8004/300000 [05:10<2:53:05, 28.12it/s]

8000 Step : 0.01567033864557743
8000 Step : tensor([0.7253], device='cuda:0', grad_fn=<DivBackward0>)
<class 'torch.Tensor'>


  3%|██                                                                         | 8039/300000 [05:11<3:08:36, 25.80it/s]


KeyboardInterrupt: 

In [ ]:
model.eval()
for i in val_list[:10]:
    v_frames,v_seq,v_edges=input_generation(pdb_dir+i)
    del v_frames
    del v_edges
    gc.collect()
    v_i_seq = data_read.encode(v_seq)
    with torch.no_grad():
        coords=model.sample(v_i_seq)
        coords=coords*10
    res = generate_res_object(v_seq,coords)
    generate_pdb(g_pdb_dir+i,res)